# Climate resilience requires equitable access to quality green energy

jobs. St. Paul is at the forefront.

Elham Ali (Beeck Center for Social Impact and Innovation at Georgetown University)  
September 19, 2024

Minnesota, particularly the City of Saint Paul, has seen a surge in climate resilience funding aimed at expanding green energy job opportunities. However, BIPOC communities remain underrepresented in these jobs and disproportionately suffer from the adverse effects of human-driven climate change.

## Background

This analysis looks at access to green energy jobs (like energy efficiency, renewable energy, and green construction) by race/ethnicity, gender, education, and income in St. Paul, Minnesota, USA.

## Questions

Here are some of the questions I will explore using different datasets:

-   How much climate resilience funding has St. Paul received?
-   What specific green jobs are being created in St. Paul (e.g., energy efficiency, renewable energy, green construction)?
-   What is the quality of these jobs? How much do they pay? What qualifications are needed (education and experience)?
-   Who is getting these jobs, based on education, race/ethnicity, gender, and income levels?

## Data Sources

The data for this project comes from:

-   The National Center for O\*NET Development
-   2023 Occupational Employment and Wage Survey
-   Urban Institute 11 elements of job quality: Clean Energy Job Quality and Education Data
-   National and local demographic data from the 2022 American Community Survey Public Use Microdata Sample (ACS PUMS)
-   US Census Bureau’s 2023 QuickFacts tool
-   Invest.gov
-   Geocorr from the Missouri Census Data Center

I will reduce each large dataset to focus only on questions related to green jobs and job quality. Please note that some datasets have already been pre-processed in Python with specific filters applied. You can find the original raw datasets in the data folder for reference.

## Analysis

I will look at each question one by one and clean the data as I go. Some datasets might need to be combined, so I will organize the data during the analysis before exploring the results.

### Load packages and libraries

In [ ]:
## For folder structure
library(here)

here() starts at /Users/elhamali/Documents/Data Projects/climate-equity-workforce

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


Attaching package: 'rlang'

The following objects are masked from 'package:purrr':

    %@%, flatten, flatten_chr, flatten_dbl, flatten_int, flatten_lgl,
    flatten_raw, invoke, splice

Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 
Highcharts (www.highcharts.com) is a Highsoft software product which is
not free for commercial and Governmental use


Attaching package: 'igraph'

The following object is masked from 'package:rlang':

    is_named

The following objects are masked from 'package:lubridate':

    %--%, union

The following objects are masked from 'package:dplyr':

    as_data_frame, groups, union

The following objects are masked from 'package:purrr':

    compose, simplify

The following object is masked from 'package:tidyr':

    crossing

The following object is masked from 'package:tibble':

    as_data_frame

The following objects are masked from 'package:stats':

    decompose, spectrum

The following object is masked from 'package:base':

    union

### 1. Climate Resilience Funding for St. Paul

> **RQ 1: How much climate resilience funding has St. Paul received?**
>
> As of June 2024, **Minnesota** received a total of \$7,101,423,527 in funding for climate resilience, while **St. Paul** received \$446,286,762. Specifically, as of January 2024, St. Paul has secured \$433,028,012 from the Bipartisan Infrastructure Law (BIL) and \$13,258,750 from the Inflation Reduction Act (IRA) for **climate resilience efforts.**
>
> St. Paul’s funding makes up **6.28%** of Minnesota’s total climate resilience funding. Nearly **95% of St. Paul’s funding i**s allocated to **transportation** **projects**, with clean energy, buildings, and manufacturing receiving **less than 2% of the total**. It’s like filling up a swimming pool with water but using only a small 8 oz glass for clean energy, buildings, and manufacturing.
>
> As of January 2024, St. Paul received **\$8,337,843 from the BIL** and **\$200,000 from the IRA** specifically for investments in clean energy, buildings, and manufacturing.

In [ ]:
# Import data
funding <- read_csv(here("processed_data", "FundingSummary.csv"))

e.g.:
  dat <- vroom(...)
  problems(dat)

Rows: 49535 Columns: 15
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (14): Agency Name, Bureau Name, Program Name, Category, Subcategory, Pro...
dbl  (1): Unique ID

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.

In [ ]:
### Convert the `Funding Amount` to numeric and handling commas in the values

funding <- funding %>%
  mutate(`Funding Amount` = as.numeric(gsub(",", "", `Funding Amount`)))

ℹ In argument: `Funding Amount = as.numeric(gsub(",", "", `Funding Amount`))`.
Caused by warning:
! NAs introduced by coercion

#### Filter for MN State and City of St. Paul

First, I will filter the dataset by State: **Minnesota**, and then narrow it down further to focus on the **City of St. Paul** and the surrounding region. Please note that St. Paul is part of the **Minneapolis-St. Paul-Bloomington, MN-WI** region, so I’ll ensure it’s included within that larger metropolitan area.

In [ ]:
# Filter for Minnesota funding
minnesota_funding <- funding %>%
  filter(State == "Minnesota")

saveRDS(minnesota_funding, here("processed_data", "minnesota_funding.rds"))

In [ ]:
# Further filter for St. Paul, considering variations in city names
st_paul_funding <- minnesota_funding %>%
  filter(str_detect(City, regex("Saint Paul|St. Paul|South St. Paul|Minneapolis--St. Paul|Minneapolis-St. Paul", ignore_case = TRUE)))

saveRDS(st_paul_funding, here("processed_data", "st_paul_funding.rds"))

# glimpse(st_paul_funding)

#### Calculate funding for MN State and City of St. Paul

In [ ]:
# Set options to avoid scientific notation
options(scipen = 999)

# Load Minnesota and St. Paul data
minnesota_funding <- readRDS(here("processed_data", "minnesota_funding.rds"))
st_paul_funding <- readRDS(here("processed_data", "st_paul_funding.rds"))

# Calculate total funding for Minnesota
total_minnesota_funding <- minnesota_funding %>%
  summarise(total_funding = sum(`Funding Amount`, na.rm = TRUE))

cat("The total amount of funding Minnesota received for climate as of June 2024 is $", 
    format(total_minnesota_funding$total_funding, big.mark = ","), "\n")

The total amount of funding Minnesota received for climate as of June 2024 is $ 7,101,423,527 

The total amount of funding St. Paul received for climate as of June 2024 is $ 446,286,762 

As of January 2024, St. Paul has been allocated $ 433,028,012  from the Bipartisan Infrastructure Law (BIL) and $ 13,258,750  from the Inflation Reduction Act (IRA).

The total amount of funding St. Paul received for 'Clean Energy, Buildings, and Manufacturing' as of June 2024 is $ 8,537,843 

As of January 2024, St. Paul has been allocated $ 8,337,843  from the Bipartisan Infrastructure Law (BIL) and $ 200,000  from the Inflation Reduction Act (IRA) to invest in 'Clean Energy, Buildings, and Manufacturing'.

As of January 2024, St. Paul has been allocated \$ 433,028,012 million from the Bipartisan Infrastructure Law (BIL) and \$ 13,258,750 from the Inflation Reduction Act (IRA) to invest in climate resilience efforts in total.

As of January 2024, St. Paul has been allocated \$ 8,337,843 million from the Bipartisan Infrastructure Law (BIL) and \$ 200,000 from the Inflation Reduction Act (IRA) to invest in ‘Clean Energy, Buildings, and Manufacturing’.

#### Calculate fraction of St. Paul’s funding from MN’s

In [ ]:
minnesota_funding <- readRDS(here("processed_data", "minnesota_funding.rds"))
st_paul_funding <- readRDS(here("processed_data", "st_paul_funding.rds"))

# Calculate total funding for Minnesota
total_minnesota_funding <- minnesota_funding %>%
  summarise(total_funding = sum(`Funding Amount`, na.rm = TRUE)) %>%
  pull(total_funding)

# Calculate total funding for St. Paul
total_st_paul_funding <- st_paul_funding %>%
  summarise(total_funding = sum(`Funding Amount`, na.rm = TRUE)) %>%
  pull(total_funding)

# Calculate the fraction of St. Paul's funding from Minnesota's total funding
fraction_st_paul <- total_st_paul_funding / total_minnesota_funding

# Output the results
cat("The fraction of St. Paul's funding from Minnesota's total funding is: ", 
    round(fraction_st_paul, 4), "\n")

The fraction of St. Paul's funding from Minnesota's total funding is:  0.0628 

This means St. Paul's funding is 6.28 % of Minnesota's total funding.

#### Visualize categories of funding for St. Paul

In [ ]:
# Group the St. Paul data by Category and calculate the total funding for each category
st_paul_category_funding <- st_paul_funding %>%
  group_by(Category) %>%
  summarise(total_funding = sum(`Funding Amount`, na.rm = TRUE)) %>%
  arrange(desc(total_funding))

colors <- brewer.pal(n = length(unique(st_paul_category_funding$Category)), "Set3")

# Create an interactive bar chart using highcharter
hchart_bar <- highchart() %>%
  hc_chart(type = "bar") %>%
  hc_xAxis(categories = st_paul_category_funding$Category, title = list(text = "Category")) %>%
  hc_yAxis(title = list(text = "Total Funding ($)"), labels = list(format = "{value:,.0f}")) %>%
  hc_add_series(name = "Total Funding", 
                data = st_paul_category_funding$total_funding, 
                colorByPoint = TRUE, 
                colors = colors) %>%
  hc_title(text = "Total Funding by Category in St. Paul") %>%
  hc_tooltip(pointFormat = "Total Funding: ${point.y:,.0f}") %>%
  hc_exporting(
    enabled = TRUE,
    buttons = list(contextButton = list(menuItems = c("downloadPNG", "downloadJPEG", "downloadSVG", "downloadPDF")))
  )

# Saving the chart as an HTML file
saveWidget(hchart_bar, file = here("graphs", "st_paul_funding_bar.html"))

A quick glance tells us that almost **95%** of St. Paul’s funding goes to transportation efforts. Clean energy, buildings and manufacturing received less than **2%** of funding.

In [ ]:
# Create an interactive pie chart using highcharter
hchart_pie <- highchart() %>%
  hc_chart(type = "pie") %>%
  hc_add_series(name = "Total Funding", 
                data = list_parse2(st_paul_category_funding %>% 
                                   mutate(name = Category, y = total_funding)), 
                colors = colors) %>%
  hc_title(text = "Total Funding by Category in St. Paul") %>%
  hc_tooltip(pointFormat = "Total Funding: ${point.y:,.0f}") %>%
  hc_plotOptions(pie = list(innerSize = '50%', dataLabels = list(enabled = TRUE))) %>%
  hc_exporting(
    enabled = TRUE,
    buttons = list(contextButton = list(menuItems = c("downloadPNG", "downloadJPEG", "downloadSVG", "downloadPDF")))
  )

saveWidget(hchart_pie, file = here("graphs", "st_paul_funding_pie.html"))

In [ ]:
## Export the funding data to CSV for graphing
write.csv(minnesota_funding, here("processed_data", "minnesota_funding.csv"), row.names = FALSE)
write.csv(st_paul_funding, here("processed_data", "st_paul_funding.csv"), row.names = FALSE)

### 2. Types of Green Jobs in St. Paul

> **RQ 2: What specific green jobs are being created in St. Paul (e.g., energy efficiency, renewable energy, green construction)?**
>
> <u>Nationally</u>
>
> There’s a total of **17,119,730 employed people** in green jobs nationally. Specifically, in **Energy Efficiency**, there are 4,928,520 (28.79 %), in **Green Construction** there are 10,624,140 (62.06 %), and in **Renewable Energy Generation** there are 1,567,070 (9.15 %).
>
> The **mean annual wage** for the occupation in U.S. dollars for green jobs is \$78,363.4, and for non-green jobs is \$73,763.67. That means green jobs pay **\$4,599.73 more** than non-green jobs **nationally**.
>
> The **mean hourly wage** for the occupation in U.S. dollars for green jobs is \$37.67547, and for non-green jobs is \$34.80. That means green jobs pay **\$2.88 more** than non-green jobs **nationally**.
>
> <u>City of Saint Paul</u>
>
> There’s a total of **214,340 employed people** in green jobs in Saint Paul. Specifically, in **Energy Efficiency**, there are 66,410 ( 30.98 %), in **Green Construction** there are 124,680 ( 58.17 %), and in **Renewable Energy Generation** there are 23,250 ( 10.85 %).
>
> The **mean annual wage** for the occupation in U.S. dollars for green jobs **in Saint Paul** is \$84,561.7, and for non-green jobs is \$77,192.53. That means green jobs in Saint Paul pay \$7,369.169 more than non-green jobs in Saint Paul.
>
> The **mean hourly wage** for the occupation in U.S. dollars for green jobs **in Saint Paul** is \$40.65, and for non-green jobs is \$36.31. That means green jobs in Saint Paul pay **\$4.35 more** than non-green jobs in Saint Paul.

#### Green jobs nationally

In [ ]:
# Import national jobs data
national_jobs <- read_csv(here("processed_data", "OWES_and_ONET-National.csv"))

Rows: 1420 Columns: 34
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (21): AREA_TITLE, PRIM_STATE, NAICS_TITLE, I_GROUP, OCC_CODE, OCC_TITLE,...
dbl  (7): AREA, AREA_TYPE, NAICS, OWN_CODE, TOT_EMP, EMP_PRSE, MEAN_PRSE
lgl  (6): JOBS_1000, LOC_QUOTIENT, PCT_TOTAL, PCT_RPT, ANNUAL, HOURLY

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.

Here, we’d want to filter to only green jobs

In [ ]:
# Convert necessary columns to numeric where needed
national_jobs <- national_jobs %>%
  mutate(
    TOT_EMP = as.numeric(TOT_EMP),
    # JOBS_1000 = as.numeric(JOBS_1000),
    # PCT_TOTAL = as.numeric(PCT_TOTAL),
    H_MEAN = as.numeric(H_MEAN),
    A_MEAN = as.numeric(A_MEAN),
    A_MEDIAN = as.numeric(A_MEDIAN),
    H_MEDIAN = as.numeric(H_MEDIAN)
  )

The first warning was:
ℹ In argument: `H_MEAN = as.numeric(H_MEAN)`.
Caused by warning:
! NAs introduced by coercion
ℹ Run `dplyr::last_dplyr_warnings()` to see the 3 remaining warnings.

# A tibble: 4 × 6
  `O*NET-SOC Sector`           TOT_EMP H_MEAN A_MEAN A_MEDIAN H_MEDIAN
  <chr>                          <dbl>  <dbl>  <dbl>    <dbl>    <dbl>
1 Energy Efficiency            4928520   43.0 89371     86355     41.5
2 Green Construction          10624140   33.9 70506.    60165     28.9
3 Renewable Energy Generation  1567070   42.3 88028.    97010     46.6
4 All                         17119730   37.7 78363.    67640     32.5

In [ ]:
# Calculate total employment and sector percentages
total_green_jobs <- final_summary %>% filter(`O*NET-SOC Sector` == "All") %>% pull(TOT_EMP)

energy_efficiency_jobs <- final_summary %>% filter(`O*NET-SOC Sector` == "Energy Efficiency") %>% pull(TOT_EMP)
green_construction_jobs <- final_summary %>% filter(`O*NET-SOC Sector` == "Green Construction") %>% pull(TOT_EMP)
renewable_energy_jobs <- final_summary %>% filter(`O*NET-SOC Sector` == "Renewable Energy Generation") %>% pull(TOT_EMP)

# Calculate the percentages
energy_efficiency_pct <- round((energy_efficiency_jobs / total_green_jobs) * 100, 2)
green_construction_pct <- round((green_construction_jobs / total_green_jobs) * 100, 2)
renewable_energy_pct <- round((renewable_energy_jobs / total_green_jobs) * 100, 2)

# Create the concatenated sentence
cat("There's a total of", format(total_green_jobs, big.mark = ",", scientific = FALSE), 
    "employed people in green jobs nationally. Specifically, in Energy Efficiency, there are", 
    format(energy_efficiency_jobs, big.mark = ",", scientific = FALSE), 
    "(", energy_efficiency_pct, "%), in Green Construction there are", 
    format(green_construction_jobs, big.mark = ",", scientific = FALSE), 
    "(", green_construction_pct, "%), and in Renewable Energy Generation there are", 
    format(renewable_energy_jobs, big.mark = ",", scientific = FALSE), 
    "(", renewable_energy_pct, "%).\n")

There's a total of 17,119,730 employed people in green jobs nationally. Specifically, in Energy Efficiency, there are 4,928,520 ( 28.79 %), in Green Construction there are 10,624,140 ( 62.06 %), and in Renewable Energy Generation there are 1,567,070 ( 9.15 %).

Let’s visualize this so it’s easier to compare across all green sectors

In [ ]:
# Convert the O*NET-SOC Sector to a factor for ordering in the chart
final_summary <- final_summary %>%
  mutate(`O*NET-SOC Sector` = factor(`O*NET-SOC Sector`, levels = c("Energy Efficiency", "Green Construction", "Renewable Energy Generation", "All")))

# Visualizing TOT_EMP across the sectors
hchart(final_summary, "column", hcaes(x = `O*NET-SOC Sector`, y = TOT_EMP)) %>%
  hc_title(text = "Total Employment by Sector") %>%
  hc_xAxis(title = list(text = "Sector")) %>%
  hc_yAxis(title = list(text = "Total Employment"), labels = list(format = "{value:,0f}")) %>%
  hc_tooltip(pointFormat = '<b>{point.y:,0f}</b>')

I’m also curious about the differences between green jobs and non-green jobs for mean hourly wage and mean annual wage.

In [ ]:
# Define green jobs as sectors related to energy and construction
green_jobs_sectors <- c("Energy Efficiency", "Renewable Energy Generation", "Green Construction")

# Add a new column to identify green and non-green jobs
national_jobs <- national_jobs %>%
  mutate(
    Job_Type = ifelse(`O*NET-SOC Sector` %in% green_jobs_sectors, "Green Jobs", "Non-Green Jobs")
  )

# Group by job type (Green vs Non-Green) and calculate mean wages
job_type_summary <- national_jobs %>%
  group_by(Job_Type) %>%
  summarize(
    H_MEAN = mean(H_MEAN, na.rm = TRUE),
    A_MEAN = mean(A_MEAN, na.rm = TRUE)
  )

# Visualizing Mean Hourly Wage (H_MEAN) for Green vs Non-Green Jobs
hchart(job_type_summary, "column", hcaes(x = Job_Type, y = H_MEAN)) %>%
  hc_title(text = "Mean Hourly Wage: Green Jobs vs Non-Green Jobs") %>%
  hc_xAxis(title = list(text = "Job Type")) %>%
  hc_yAxis(title = list(text = "Mean Hourly Wage (USD)")) %>%
  hc_tooltip(pointFormat = '<b>{point.y:.2f} USD</b>')

In [ ]:
# Summarizing core findings nationally

# Extract green and non-green job wage data
green_wages <- job_type_summary %>% filter(Job_Type == "Green Jobs")
non_green_wages <- job_type_summary %>% filter(Job_Type == "Non-Green Jobs")

# Calculate the difference between green and non-green jobs
difference_annual <- green_wages$A_MEAN - non_green_wages$A_MEAN
difference_hourly <- green_wages$H_MEAN - non_green_wages$H_MEAN

# Format and print the sentences
cat("The mean annual wage for the occupation in U.S. dollars for green jobs is $", 
    format(green_wages$A_MEAN, big.mark = ",", scientific = FALSE), 
    ", and for non-green jobs is $", 
    format(non_green_wages$A_MEAN, big.mark = ",", scientific = FALSE), 
    ". That means green jobs pay $", 
    format(abs(difference_annual), big.mark = ",", scientific = FALSE), 
    ifelse(difference_annual > 0, " more", " less"), 
    " than non-green jobs nationally.\n", sep = "")

The mean annual wage for the occupation in U.S. dollars for green jobs is $78,363.4, and for non-green jobs is $73,763.67. That means green jobs pay $4,599.726 more than non-green jobs nationally.

The mean hourly wage for the occupation in U.S. dollars for green jobs is $37.67547, and for non-green jobs is $34.79641. That means green jobs pay $2.879063 more than non-green jobs nationally.

I’d like to see a word cloud of different job titles for each sector

In [ ]:
# Filter the dataset for green jobs only
green_jobs <- national_jobs %>%
  filter(`O*NET-SOC Sector` %in% c("Energy Efficiency", "Renewable Energy Generation", "Green Construction"))

# Extract job titles and count their occurrences
job_titles <- green_jobs %>%
  count(OCC_TITLE, sort = TRUE)

# Create a word cloud using highcharter
hchart(
  job_titles, 
  "wordcloud", 
  hcaes(name = OCC_TITLE, weight = n)
) %>%
  hc_title(text = "Word Cloud of Green Job Titles")

Now let’s create separate word clouds for each of the green sectors ("Energy Efficiency", "Renewable Energy Generation", and "Green Construction").

In [ ]:
# Filter the dataset for each sector
energy_efficiency_jobs <- national_jobs %>%
  filter(`O*NET-SOC Sector` == "Energy Efficiency")

renewable_energy_jobs <- national_jobs %>%
  filter(`O*NET-SOC Sector` == "Renewable Energy Generation")

green_construction_jobs <- national_jobs %>%
  filter(`O*NET-SOC Sector` == "Green Construction")

# Create a function to generate word clouds
generate_wordcloud <- function(data, sector_name) {
  job_titles <- data %>%
    count(OCC_TITLE, sort = TRUE)
  
  hchart(
    job_titles, 
    "wordcloud", 
    hcaes(name = OCC_TITLE, weight = n)
  ) %>%
    hc_title(text = paste("Word Cloud of", sector_name, "Job Titles"))
}

# Generate word cloud for Energy Efficiency
energy_efficiency_wordcloud <- generate_wordcloud(energy_efficiency_jobs, "Energy Efficiency")

# Generate word cloud for Renewable Energy Generation
renewable_energy_wordcloud <- generate_wordcloud(renewable_energy_jobs, "Renewable Energy Generation")

# Generate word cloud for Green Construction
green_construction_wordcloud <- generate_wordcloud(green_construction_jobs, "Green Construction")

# Display the word clouds
energy_efficiency_wordcloud

Let’s export for graphing

In [ ]:
## Export the national jobs data to CSV for graphing
write.csv(national_jobs, here("processed_data", "national_jobs.csv"), row.names = FALSE)

# Export the job_type_summary dataset to CSV for graphing
write.csv(job_type_summary, here("processed_data", "national_job_type_summary.csv"), row.names = FALSE)

#### Green jobs in St. Paul

In [ ]:
# Import St. Paul jobs data
st_paul_jobs <- read_csv(here("processed_data", "OWES_and_ONET-St_Paul.csv"))

Rows: 742 Columns: 34
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (26): AREA_TITLE, PRIM_STATE, NAICS_TITLE, I_GROUP, OCC_CODE, OCC_TITLE,...
dbl  (4): AREA, AREA_TYPE, NAICS, OWN_CODE
lgl  (4): PCT_TOTAL, PCT_RPT, ANNUAL, HOURLY

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.

In [ ]:
# Convert necessary columns to numeric where needed
st_paul_jobs <- st_paul_jobs %>%
  mutate(
    TOT_EMP = as.numeric(TOT_EMP),
    H_MEAN = as.numeric(H_MEAN),
    A_MEAN = as.numeric(A_MEAN),
    A_MEDIAN = as.numeric(A_MEDIAN),
    H_MEDIAN = as.numeric(H_MEDIAN)
  )

The first warning was:
ℹ In argument: `TOT_EMP = as.numeric(TOT_EMP)`.
Caused by warning:
! NAs introduced by coercion
ℹ Run `dplyr::last_dplyr_warnings()` to see the 4 remaining warnings.

# A tibble: 4 × 6
  `O*NET-SOC Sector`          TOT_EMP H_MEAN A_MEAN A_MEDIAN H_MEDIAN
  <chr>                         <dbl>  <dbl>  <dbl>    <dbl>    <dbl>
1 Energy Efficiency             66410   45.5 94669.    98740     47.5
2 Green Construction           124680   37.9 78809.    75800     36.4
3 Renewable Energy Generation   23250   44.7 92991.    99690     47.9
4 All                          214340   40.7 84562.    82170     39.5

There's a total of 214,340 employed people in green jobs in Saint Paul. Specifically, in Energy Efficiency, there are 66,410 ( 30.98 %), in Green Construction there are 124,680 ( 58.17 %), and in Renewable Energy Generation there are 23,250 ( 10.85 %).

In [ ]:
# Convert the O*NET-SOC Sector to a factor for ordering in the chart for St. Paul
final_summary_st_paul <- final_summary_st_paul %>%
  mutate(`O*NET-SOC Sector` = factor(`O*NET-SOC Sector`, levels = c("Energy Efficiency", "Green Construction", "Renewable Energy Generation", "All")))

# Visualizing TOT_EMP across the sectors for St. Paul
hchart(final_summary_st_paul, "column", hcaes(x = `O*NET-SOC Sector`, y = TOT_EMP)) %>%
  hc_title(text = "Total Employment by Sector in Saint Paul") %>%
  hc_xAxis(title = list(text = "Sector")) %>%
  hc_yAxis(title = list(text = "Total Employment"), labels = list(format = "{value:,0f}")) %>%
  hc_tooltip(pointFormat = '<b>{point.y:,0f}</b>')

The mean annual wage for the occupation in U.S. dollars for green jobs in Saint Paul is $84,561.7, and for non-green jobs is $77,192.53. That means green jobs in Saint Paul pay $7,369.169 more than non-green jobs in Saint Paul.

The mean hourly wage for the occupation in U.S. dollars for green jobs in Saint Paul is $40.65447, and for non-green jobs is $36.30688. That means green jobs in Saint Paul pay $4.347591 more than non-green jobs in Saint Paul.

I’d like to see a word cloud of different job titles for each sector in St. Paul

In [ ]:
# Filter the dataset for green jobs only in St. Paul
green_jobs_st_paul <- st_paul_jobs %>%
  filter(`O*NET-SOC Sector` %in% c("Energy Efficiency", "Renewable Energy Generation", "Green Construction"))

# Extract job titles and count their occurrences in St. Paul
job_titles_st_paul <- green_jobs_st_paul %>%
  count(OCC_TITLE, sort = TRUE)

# Create a word cloud for green jobs in St. Paul using highcharter
hchart(
  job_titles_st_paul, 
  "wordcloud", 
  hcaes(name = OCC_TITLE, weight = n)
) %>%
  hc_title(text = "Word Cloud of Green Job Titles in Saint Paul")

Now let’s create separate word clouds for each of the green sectors ("Energy Efficiency", "Renewable Energy Generation", and "Green Construction") for St. Paul

In [ ]:
# Filter the dataset for each sector in St. Paul
energy_efficiency_jobs_st_paul <- st_paul_jobs %>%
  filter(`O*NET-SOC Sector` == "Energy Efficiency")

renewable_energy_jobs_st_paul <- st_paul_jobs %>%
  filter(`O*NET-SOC Sector` == "Renewable Energy Generation")

green_construction_jobs_st_paul <- st_paul_jobs %>%
  filter(`O*NET-SOC Sector` == "Green Construction")

# Create a function to generate word clouds for St. Paul sectors
generate_wordcloud_st_paul <- function(data, sector_name) {
  job_titles <- data %>%
    count(OCC_TITLE, sort = TRUE)
  
  hchart(
    job_titles, 
    "wordcloud", 
    hcaes(name = OCC_TITLE, weight = n)
  ) %>%
    hc_title(text = paste("Word Cloud of", sector_name, "Job Titles in Saint Paul"))
}

# Generate word cloud for Energy Efficiency in St. Paul
energy_efficiency_wordcloud_st_paul <- generate_wordcloud_st_paul(energy_efficiency_jobs_st_paul, "Energy Efficiency")

# Generate word cloud for Renewable Energy Generation in St. Paul
renewable_energy_wordcloud_st_paul <- generate_wordcloud_st_paul(renewable_energy_jobs_st_paul, "Renewable Energy Generation")

# Generate word cloud for Green Construction in St. Paul
green_construction_wordcloud_st_paul <- generate_wordcloud_st_paul(green_construction_jobs_st_paul, "Green Construction")

# Display the word clouds for St. Paul
energy_efficiency_wordcloud_st_paul

Let’s export for graphing

In [ ]:
# Export the St. Paul jobs data to CSV for graphing
write.csv(st_paul_jobs, here("processed_data", "st_paul_jobs.csv"), row.names = FALSE)

# Export the job_type_summary dataset for St. Paul to CSV for graphing
write.csv(job_type_summary_st_paul, here("processed_data", "st_paul_job_type_summary.csv"), row.names = FALSE)

### 3. Quality, Pay, and Qualifications of Green Jobs

> **RQ 3: What is the quality of these jobs? How much do they pay? What qualifications are needed (education and experience)?**
>
> TBD

### 4. Demographics of Green Job Recipients

> **RQ 4: Who is getting these jobs, based on education, race/ethnicity, gender, and income levels?**
>
> Of the more than **303,820 people** who live in **St. Paul**, **50.5%** are women, which is aligned with the national average. The majority of residents in St. Paul are **white** (54.3%). **Black or African American people** (15.6%) make up the largest community of color in the city. Other **communities of color,** including Asian (18.4%), Alaska and Native American (0.7%), Hispanic or Latino (8.6%) and Two or More Races (7.8%), make up about 41.5% of the population. Around **42.8% of people aged 25 and older have a bachelor’s degree** in St. Paul, which is higher than the national rate of 34% but lower than Minneapolis’ 54%.

In the following figures, we describe the challenges women and people of color in Minneapolis are likely to face in equitably accessing the jobs that may be created through BIL and IRA funding in three specific sectors: energy efficiency, renewable-energy generation, and green construction.

**Sources**: Demographic data from 2022 American Community Survey Public Use Microdata Sample (ACS PUMS); total employment from 2022 Occupational Employment and Wage Survey (OEWS); occupational job quality ratings from [Job Quality and Race and Gender Equity](https://www.urban.org/research/publication/job-quality-and-race-and-gender-equity); occupation codes from [O\*NET](https://www.onetonline.org/).